In [ ]:
import pickle
import sys
from zoneinfo import ZoneInfo
sys.path.append("../")

from dotenv import load_dotenv
load_dotenv()
import geopandas as gpd
import importlib
import copy
import logging
import contextily as cx
import gtfs_kit as gk
import fastsim as fsim
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import lightning.pytorch as pl
import rasterio as rio
from rasterio.plot import show
import seaborn as sns
import shapely
import statsmodels.api as sm
from torch.utils.data import DataLoader

from openbustools import plotting, spatial, standardfeeds
from openbustools.traveltime import data_loader, model_utils
from openbustools.drivecycle import trajectory
from openbustools.drivecycle.physics import conditions, energy, vehicle

In [ ]:
epsg = 32148
timezone = "America/Los_Angeles"
coord_ref_center = [386910,69022]
apply_filter = True
chop_n = 10
point_sep_m = 300
dem_file = Path("..","data","kcm_spatial","usgs10m_dem_32148.tif")
phone_trajectory_folders = [x for x in Path("..","data","kcm_sensor","match_realtime").glob("*") if x.is_dir()]
realtime_folder = Path("..","data","kcm_sensor_realtime")

model_folder = "../logs"
run_label = "/kcm"
model_type = "GRU"
fold_num = 0
version = "version_3"
model = model_utils.load_model(model_folder, run_label, model_type, fold_num, version=version)
model.eval()

### Phone, Realtime, Receiver Comparison

In [ ]:
# Load phone/realtime data as trajectories
phone_trajectories = [standardfeeds.get_phone_trajectory(x, timezone=timezone, epsg=epsg, coord_ref_center=coord_ref_center, apply_filter=apply_filter, chop_n=chop_n) for x in phone_trajectory_folders]
phone_realtime_trajectories = [standardfeeds.get_realtime_trajectory(x, realtime_folder=realtime_folder, dem_file=dem_file) for x in phone_trajectories]

In [ ]:
# Run fastsim energy analysis for each trajectory
energy_results = []
# veh = fsim.vehicle.Vehicle.from_vehdb(17, veh_file=Path("..", "data", "FASTSim_py_veh_db.csv")) # Chevy Bolt
# veh = fsim.vehicle.Vehicle.from_vehdb(52, veh_file=Path("..", "data", "FASTSim_py_veh_db.csv")) # Ford Lightning
veh = fsim.vehicle.Vehicle.from_vehdb(63, veh_file=Path("..", "data", "FASTSim_py_veh_db.csv")) # New Flyer XE40
print(f"Vehicle: {veh.scenario_name}\n")

# Baseline energy analysis for standard cycle
cycle_udds = fsim.cycle.Cycle.from_file("udds")
cycle_udds = fsim.cycle.Cycle.from_dict(fsim.cycle.resample(cycle_udds.to_dict(), new_dt=1))
sim_drive_udds = fsim.simdrive.SimDrive(cycle_udds, veh)
sim_drive_udds.sim_drive()

# Phone/realtime energy consumption
efficiencies = []
for i, (traj_phone, traj_realtime) in enumerate(zip(phone_trajectories, phone_realtime_trajectories)):
    # Energy analysis for phone
    cycle_phone = traj_phone.to_fastsim_cycle()
    cycle_phone = fsim.cycle.Cycle.from_dict(fsim.cycle.resample(cycle_phone, new_dt=1))
    sim_drive_phone = fsim.simdrive.SimDrive(cycle_phone, veh)
    sim_drive_phone.sim_drive()
    # Energy analysis for realtime
    cycle_realtime = traj_realtime.to_fastsim_cycle()
    cycle_realtime = fsim.cycle.Cycle.from_dict(fsim.cycle.resample(cycle_realtime, new_dt=1))
    sim_drive_realtime = fsim.simdrive.SimDrive(cycle_realtime, veh)
    sim_drive_realtime.sim_drive()
    energy_results.append({"phone": (cycle_phone, sim_drive_phone), "realtime": (cycle_realtime, sim_drive_realtime)})
    efficiencies.append((1 / sim_drive_phone.electric_kwh_per_mi, 1 / sim_drive_realtime.electric_kwh_per_mi))

print(f"Efficiencies:")
print(f"Altoona New Flyer XE40 UDDS: {0.57} mi/kWh")
print(f"UDDS: {1 / sim_drive_udds.battery_kwh_per_mi:.2f} mi/kWh")
print(f"Avg. Phone: {np.mean([x[0] for x in efficiencies]):.2f} mi/kWh")
print(f"Avg. Realtime: {np.mean([x[1] for x in efficiencies]):.2f} mi/kWh\n")
print(f"RMSE Phone/Realtime: {np.sqrt(np.mean([(x-y)**2 for x,y in efficiencies])):.2f} mi/kWh")
fig, axes = plt.subplots(1, 1, figsize=(15, 5))
sns.lineplot(x=sim_drive_udds.cyc.time_s, y=sim_drive_udds.cyc.mph, label="UDDS")
sns.lineplot(x=sim_drive_phone.cyc.time_s, y=sim_drive_phone.cyc.mph, label="Phone")
sns.lineplot(x=sim_drive_realtime.cyc.time_s, y=sim_drive_realtime.cyc.mph, label="Realtime")
plt.xlabel("Time (s)")
plt.ylabel("Speed (mph)")

In [ ]:
subset = energy_results[0:4]

fig, axes = plt.subplots(9, len(subset), figsize=(18,20))
fig.tight_layout()
for i, res in enumerate(subset):
    res_cycle_phone, res_simdrive_phone = res["phone"]
    res_cycle_realtime, res_simdrive_realtime = res["realtime"]
    ax = axes[0,i]
    ax.plot(res_simdrive_phone.cyc.time_s, res_simdrive_phone.mph_ach, label="Phone")
    ax.plot(res_simdrive_realtime.cyc.time_s, res_simdrive_realtime.mph_ach, label="Realtime")
    ax = axes[1,i]
    ax.plot(res_simdrive_phone.cyc.time_s, res_simdrive_phone.cyc.grade, label="Phone")
    ax.plot(res_simdrive_realtime.cyc.time_s, res_simdrive_realtime.cyc.grade, label="Realtime")
    ax = axes[2,i]
    ax.plot(res_simdrive_phone.cyc.time_s, res_simdrive_phone.ess_kw_out_ach, label="Phone")
    ax.plot(res_simdrive_realtime.cyc.time_s, res_simdrive_realtime.ess_kw_out_ach, label="Realtime")
    ax = axes[3,i]
    ax.plot(res_simdrive_phone.cyc.time_s, res_simdrive_phone.accel_kw, label="Phone")
    ax.plot(res_simdrive_realtime.cyc.time_s, res_simdrive_realtime.accel_kw, label="Realtime")
    ax = axes[4,i]
    ax.plot(res_simdrive_phone.cyc.time_s, res_simdrive_phone.rr_kw, label="Phone")
    ax.plot(res_simdrive_realtime.cyc.time_s, res_simdrive_realtime.rr_kw, label="Realtime")
    ax = axes[5,i]
    ax.plot(res_simdrive_phone.cyc.time_s, res_simdrive_phone.ess_loss_kw, label="Phone")
    ax.plot(res_simdrive_realtime.cyc.time_s, res_simdrive_realtime.ess_loss_kw, label="Realtime")
    ax = axes[6,i]
    ax.plot(res_simdrive_phone.cyc.time_s, res_simdrive_phone.ascent_kw, label="Phone")
    ax.plot(res_simdrive_realtime.cyc.time_s, res_simdrive_realtime.ascent_kw, label="Realtime")
    ax = axes[7,i]
    ax.plot(res_simdrive_phone.cyc.time_s, res_simdrive_phone.drag_kw, label="Phone")
    ax.plot(res_simdrive_realtime.cyc.time_s, res_simdrive_realtime.drag_kw, label="Realtime")
    ax = axes[8,i]
    ax.plot(res_simdrive_phone.cyc.time_s, res_simdrive_phone.aux_in_kw, label="Phone")
    ax.plot(res_simdrive_realtime.cyc.time_s, res_simdrive_realtime.aux_in_kw, label="Realtime")

axes[0,0].legend()
axes[0,0].set_xlabel("Time [s]")

axes[0,0].set_title("Sample Trip 1")
axes[0,1].set_title("Sample Trip 2")
axes[0,2].set_title("Sample Trip 3")
axes[0,3].set_title("Sample Trip 3")

axes[0,0].set_ylabel("Speed [MPH]")
axes[1,0].set_ylabel("Grade [%/100]")
axes[2,0].set_ylabel("ESS Total Power [kW]")
axes[3,0].set_ylabel("Acceleration Power [kW]")
axes[4,0].set_ylabel("RR Power [kW]")
axes[5,0].set_ylabel("ESS Loss Power [kW]")
axes[6,0].set_ylabel("Ascent Power [kW]")
axes[7,0].set_ylabel("Drag Power [kW]")
axes[8,0].set_ylabel("Aux Power [kW]")
plt.show()

### Mean Realtime, Model Predicted Comparison

In [ ]:
# Load a static feed and break each shape into regularly spaced points
static_file = Path("..", "data", "kcm_static", "2023_03_18")
static_feed = gk.read_feed(static_file, dist_units="km")
route_shape_points = standardfeeds.segmentize_route_shapes(static_feed, epsg=epsg, point_sep_m=point_sep_m)

# Load set of realtime data to aggregate to route shapes; join it to static feed ids
realtime_files = [Path("..", "data", "kcm_realtime", "processed", "analysis", f"2023_03_2{x}.pkl") for x in np.arange(5)]
realtime_data = pd.concat([pd.read_pickle(x) for x in realtime_files])
realtime_data = pd.merge(realtime_data[['calc_speed_m_s', 'x', 'y', 'trip_id']], static_feed.trips[['trip_id', 'shape_id']], on='trip_id')
realtime_data = gpd.GeoDataFrame(realtime_data, geometry=gpd.points_from_xy(realtime_data.x, realtime_data.y), crs=epsg)

# Group realtime data by shape and find closest static point in that shape for each observation
route_shape_metrics = realtime_data.groupby('shape_id').apply(lambda x: gpd.sjoin_nearest(x, route_shape_points[x.name]), include_groups=False)
route_shape_metrics = route_shape_metrics.drop(columns=['shape_id']).reset_index()
route_shape_metrics = route_shape_metrics.groupby(['shape_id', 'seq_id'], as_index=False).agg({
    'calc_speed_m_s': ['mean', 'std'],
    'trip_id': 'count',
    'geometry': 'first'
})

# Aggregate observations to the shapes
route_shape_metrics = {
    'geometry': route_shape_metrics[('geometry', 'first')],
    'speed_mean': route_shape_metrics[('calc_speed_m_s', 'mean')],
    'speed_std': route_shape_metrics[('calc_speed_m_s', 'std')],
    'count': route_shape_metrics[('trip_id', 'count')],
    'shape_id': route_shape_metrics['shape_id'],
    'seq_id': route_shape_metrics['seq_id']
}
route_shape_metrics = gpd.GeoDataFrame(route_shape_metrics, crs=epsg)
route_shape_metrics.head()

In [ ]:
# Subset of shape_ids for testing
plot_routes = route_shape_metrics.sample(4, random_state=42)['shape_id'].to_numpy()
plot_df = route_shape_metrics[route_shape_metrics['shape_id'].isin(plot_routes)].copy()
plot_df_groups = {k: v for k, v in plot_df.groupby('shape_id')}

In [ ]:
# Turning regularly spaced points w/speeds into calculated distances and times
# Important to base distance on sequence id because some points do not have observations
for k,df in plot_df_groups.items():
    plot_df_groups[k]['cumul_dist_m'] = df['seq_id'] * point_sep_m
    plot_df_groups[k]['calc_dist_m'] = df['cumul_dist_m'].diff().fillna(0)
    plot_df_groups[k]['calc_time_s'] = df['calc_dist_m'] / df['speed_mean']
    plot_df_groups[k]['cumul_time_s'] = df['calc_time_s'].cumsum()
    plot_df_groups[k]['speed_std'] = df['speed_std'].bfill().ffill()

In [ ]:
# Overview of the routes
fig, axes = plt.subplots(1,len(plot_df_groups), figsize=(30,5))
fig.tight_layout()
for i, (k,df) in enumerate(plot_df_groups.items()):
    ax = axes[i]
    ax.plot(df['cumul_dist_m'], df['speed_mean'], label="Mean Speed")
    ax.fill_between(df['cumul_dist_m'], df['speed_mean'] - df['speed_std'], df['speed_mean'] + df['speed_std'], alpha=0.2, label="Speed Std")
    ax.set_xlabel("Distance [m]")
    ax.set_ylabel("Speed [m/s]")
    ax.set_title(f"Shape {k}")
    ax.legend()

In [ ]:
# Map of the routes
fig, axes = plt.subplots(1,len(plot_df_groups), figsize=(30,5))
fig.tight_layout()
for i, (k,df) in enumerate(plot_df_groups.items()):
    ax = axes[i]
    df.plot(ax=ax, column='speed_mean', cmap='plasma', legend=True)
    cx.add_basemap(ax=ax, crs=plot_df.crs.to_string(), alpha=0.6, source=cx.providers.MapBox(accessToken=os.getenv(key="MAPBOX_TOKEN")))
    ax.set_title(f"Shape {k}")

In [ ]:
# Create aggregated trajectory and predicted trajectory for each shape
agg_trajectories = []
pred_trajectories = []
for shape_id, df in plot_df_groups.items():
    agg_traj = trajectory.Trajectory(
        point_attr={
            "lon": df.to_crs(4326).geometry.x.to_numpy(),
            "lat": df.to_crs(4326).geometry.y.to_numpy(),
            "measured_speed_m_s": df.speed_mean.to_numpy(),
            "cumul_time_s": df.cumul_time_s.to_numpy()
        },
        traj_attr={'shape_id': shape_id},
        coord_ref_center=coord_ref_center,
        epsg=epsg,
        apply_filter=apply_filter,
        dem_file=dem_file
    )
    agg_trajectories.append(agg_traj)
    pred_traj = trajectory.Trajectory(
        point_attr={
            "lon": df.to_crs(4326).geometry.x.to_numpy(),
            "lat": df.to_crs(4326).geometry.y.to_numpy(),
            "measured_speed_m_s": df.speed_mean.to_numpy(),
            "cumul_time_s": df.cumul_time_s.to_numpy()
        },
        traj_attr={'shape_id': shape_id},
        coord_ref_center=coord_ref_center,
        epsg=epsg,
        apply_filter=apply_filter,
        dem_file=dem_file
    )
    pred_traj.pred_time_s = trajectory.predict_trajectory_times([pred_traj], model)[0]
    pred_trajectories.append(pred_traj)

In [ ]:
energy_results = []
efficiencies = []

veh = fsim.vehicle.Vehicle.from_vehdb(63, veh_file=Path("..", "data", "FASTSim_py_veh_db.csv")) # New Flyer XE40
print(f"Vehicle: {veh.scenario_name}\n")

for i, (traj_agg, traj_pred) in enumerate(zip(agg_trajectories, pred_trajectories)):
    # Energy analysis for aggregated
    cycle_agg = traj_agg.to_fastsim_cycle()
    cycle_agg = fsim.cycle.Cycle.from_dict(fsim.cycle.resample(cycle_agg, new_dt=1))
    sim_drive_agg = fsim.simdrive.SimDrive(cycle_agg, veh)
    sim_drive_agg.sim_drive()
    # Energy analysis for predicted
    cycle_pred = traj_pred.to_fastsim_cycle()
    cycle_pred = fsim.cycle.Cycle.from_dict(fsim.cycle.resample(cycle_pred, new_dt=1))
    sim_drive_pred = fsim.simdrive.SimDrive(cycle_pred, veh)
    sim_drive_pred.sim_drive()
    energy_results.append({"agg": (cycle_agg, sim_drive_agg), "pred": (cycle_pred, sim_drive_pred)})
    efficiencies.append((1 / sim_drive_agg.electric_kwh_per_mi, 1 / sim_drive_pred.electric_kwh_per_mi))

print(f"Efficiencies:")
print(f"Avg. Agg: {np.mean([x[0] for x in efficiencies]):.2f} mi/kWh")
print(f"Avg. Pred: {np.mean([x[1] for x in efficiencies]):.2f} mi/kWh\n")
print(f"RMSE Agg/Pred: {np.sqrt(np.mean([(x-y)**2 for x,y in efficiencies])):.2f} mi/kWh")

In [ ]:
subset = energy_results[0:4]

fig, axes = plt.subplots(9, len(subset), figsize=(18,20))
fig.tight_layout()
for i, res in enumerate(subset):
    res_cycle_agg, res_simdrive_agg = res["agg"]
    res_cycle_pred, res_simdrive_pred = res["pred"]
    ax = axes[0,i]
    ax.plot(res_simdrive_agg.cyc.time_s, res_simdrive_agg.mph_ach, label="Agg")
    ax.plot(res_simdrive_pred.cyc.time_s, res_simdrive_pred.mph_ach, label="Pred")
    ax = axes[1,i]
    ax.plot(res_simdrive_agg.cyc.time_s, res_simdrive_agg.cyc.grade, label="Agg")
    ax.plot(res_simdrive_pred.cyc.time_s, res_simdrive_pred.cyc.grade, label="Pred")
    ax = axes[2,i]
    ax.plot(res_simdrive_agg.cyc.time_s, res_simdrive_agg.ess_kw_out_ach, label="Agg")
    ax.plot(res_simdrive_pred.cyc.time_s, res_simdrive_pred.ess_kw_out_ach, label="Pred")
    ax = axes[3,i]
    ax.plot(res_simdrive_agg.cyc.time_s, res_simdrive_agg.accel_kw, label="Agg")
    ax.plot(res_simdrive_pred.cyc.time_s, res_simdrive_pred.accel_kw, label="Pred")
    ax = axes[4,i]
    ax.plot(res_simdrive_agg.cyc.time_s, res_simdrive_agg.rr_kw, label="Agg")
    ax.plot(res_simdrive_pred.cyc.time_s, res_simdrive_pred.rr_kw, label="Pred")
    ax = axes[5,i]
    ax.plot(res_simdrive_agg.cyc.time_s, res_simdrive_agg.ess_loss_kw, label="Agg")
    ax.plot(res_simdrive_pred.cyc.time_s, res_simdrive_pred.ess_loss_kw, label="Pred")
    ax = axes[6,i]
    ax.plot(res_simdrive_agg.cyc.time_s, res_simdrive_agg.ascent_kw, label="Agg")
    ax.plot(res_simdrive_pred.cyc.time_s, res_simdrive_pred.ascent_kw, label="Pred")
    ax = axes[7,i]
    ax.plot(res_simdrive_agg.cyc.time_s, res_simdrive_agg.drag_kw, label="Agg")
    ax.plot(res_simdrive_pred.cyc.time_s, res_simdrive_pred.drag_kw, label="Pred")
    ax = axes[8,i]
    ax.plot(res_simdrive_agg.cyc.time_s, res_simdrive_agg.aux_in_kw, label="Agg")
    ax.plot(res_simdrive_pred.cyc.time_s, res_simdrive_pred.aux_in_kw, label="Pred")

axes[0,0].legend()
axes[0,0].set_xlabel("Time [s]")

axes[0,0].set_title("Sample Shape 1")
axes[0,1].set_title("Sample Shape 2")
axes[0,2].set_title("Sample Shape 3")
axes[0,3].set_title("Sample Shape 3")

axes[0,0].set_ylabel("Speed [MPH]")
axes[1,0].set_ylabel("Grade [%/100]")
axes[2,0].set_ylabel("ESS Total Power [kW]")
axes[3,0].set_ylabel("Acceleration Power [kW]")
axes[4,0].set_ylabel("RR Power [kW]")
axes[5,0].set_ylabel("ESS Loss Power [kW]")
axes[6,0].set_ylabel("Ascent Power [kW]")
axes[7,0].set_ylabel("Drag Power [kW]")
axes[8,0].set_ylabel("Aux Power [kW]")
plt.show()

### Expand Predicted to Full Network

### Compare Phone Sensor Kinematics

In [ ]:
# fig, axes = plt.subplots(1,1,figsize=(8,4))
# phone_traj.gdf['calc_dist_m'].cumsum().plot(ax=axes)
# phone_traj.gdf['measured_speed_m_s'].cumsum().plot(ax=axes)
# phone_traj.gdf['measured_accel_m_s2'].cumsum().cumsum().plot(ax=axes)
# axes.legend(['gps','integrated speedometer','integrated accelerometer'])
# axes.set_title("Total Distance (m)")

In [ ]:
# fig, axes = plt.subplots(1,1,figsize=(8,4))
# phone_traj.gdf['calc_dist_m'].plot(ax=axes)
# phone_traj.gdf['measured_speed_m_s'].plot(ax=axes)
# phone_traj.gdf['measured_accel_m_s2'].cumsum().plot(ax=axes)
# axes.legend(['rate of change gps','speedometer','integrated accelerometer'])
# axes.set_title("Velocity (m/s)")

In [ ]:
# fig, axes = plt.subplots(1,1,figsize=(8,4))
# phone_traj.gdf['calc_dist_m'].diff().clip(-1,1).plot(ax=axes)
# phone_traj.gdf['measured_speed_m_s'].diff().plot(ax=axes)
# phone_traj.gdf['measured_accel_m_s2'].plot(ax=axes)
# axes.legend(['rate of change gps','rate of change speedometer','accelerometer'])
# axes.set_title("Acceleration (m/s^2)")

### Distance Along GTFS Shape

In [ ]:
# # GTFS shapes
# shape_lookup = standardfeeds.get_gtfs_shapes_lookup(f"../data/kcm_gtfs/{static_date}/")
# shapes = standardfeeds.get_gtfs_shapes(f"../data/kcm_gtfs/{static_date}/").to_crs("EPSG:32148")
# shapes.plot()

# route_ids = pd.unique(data_gtfs[(data_gtfs['route_short_name']==short_name) & (data_gtfs['direction_id']==0)].route_id)
# phone_shape = shapes[(shapes['route_id'].isin(route_ids)) & (shapes['direction_id']==0) & (shapes['service_id']==21133)]

# # Get one shape to work with
# sample_service_id, sample_route_id, sample_direction_id = data_gtfsrt.groupby(['service_id','route_id','direction_id']).count().index[0]
# print(sample_service_id, sample_route_id, sample_direction_id)

# # GTFS-RT
# sample_realtime = data_gtfsrt[(data_gtfsrt['service_id']==sample_service_id) & (data_gtfsrt['route_id']==sample_route_id) & (data_gtfsrt['direction_id']==sample_direction_id)].copy()

# # Shape
# sample_shape = shapes[(shapes['service_id']==sample_service_id) & (shapes['route_id']==sample_route_id) & (shapes['direction_id']==sample_direction_id)].copy()
# sample_shape.plot()

# # Get distance along shape
# sample_realtime['dist_along_line'] = sample_realtime['geometry'].apply(lambda pt: shapely.line_locate_point(sample_shape.geometry, pt))
# # sample_static['dist_along_line'] = sample_static['geometry'].apply(lambda pt: shapely.line_locate_point(sample_shape.geometry, pt))

# # Also get a timestamp column on the samples
# sample_realtime['t'] = pd.to_datetime(sample_realtime['locationtime'], unit='s')
# sample_realtime = sample_realtime.set_index('t')